## **Import Libraries**

In [1]:
import GetOldTweets3 as got
import time
from datetime import datetime, date, timedelta
import pandas as pd
import numpy as np
import os
import tweepy as tw

In [2]:
pip install tweepy

Note: you may need to restart the kernel to use updated packages.


## **Get The Tweet and Usernames**

In [8]:
tweets = []
user = []
def DownloadTweets(SinceDate, UntilDate, query, sleep=900, maxtweet=0) :
    #create a list of day numbers
    since = datetime.strptime(SinceDate, '%Y-%m-%d')
    
#   Days is the amount of days we are pulling data from. This is subtracting the SinceDate from the UntilDate
    days = list(range(0, (datetime.strptime(UntilDate, '%Y-%m-%d') - datetime.strptime(SinceDate, '%Y-%m-%d')).days+1))

#   This loop will run for the number of days above eg. days is 50 --> this loop will run 50 times 
    for day in days:
        init = got.manager.TweetCriteria().setQuerySearch(query).setSince((since + timedelta(days=day)).strftime('%Y-%m-%d')).setUntil((since+ timedelta(days=day+1)).strftime('%Y-%m-%d')).setMaxTweets(maxtweet)
        get = got.manager.TweetManager.getTweets(init)
        
        # Appending a list of the tweets to an empty list above
        tweets.append([[tweet.text] for tweet in get])
        
        # Appending a list of usernames to an empty list above
        user.append([use.username for use in get])
#         print("day", day+1, "of", len(days), "completed")
     
    time.sleep(sleep)

    return 

since = "2020-09-03"  #SinceDate
until = "2020-09-04"  #UntilDate

q_list = [' my power went out ', 'I just lost power', 'my power is out', 
          ' I have a power outage ', 'my power is down']

# For every query in q_list this function will run and add the tweets and usernames to the empty lists
for n in q_list: 
    tweets2 = DownloadTweets(since, until, query=n, maxtweet=3, sleep=15)
    

# ================================================================================================================================    
# This functions will splits the pulled user name into a list of strings in a list
def split(users):
    fin_user = []
    for xi in users:
        for n in xi:
            fin_user.append(n.split(","))
            
    return fin_user

users = split(user) # This is calling the function

hmm = [tweet[0] for sublist in tweets for tweet in sublist] # This gets us the string of the tweet before it was a list of list of lists


fin_user = [''.join(ele) for ele in users] # This gets rid of the [] around the individual stringed usernames
df = pd.DataFrame({'tweets': hmm, 
                   'user': fin_user
                  })

df

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



An error occured during an HTTP request: HTTP Error 429: Too Many Requests
Try to open in browser: https://twitter.com/search?q=%20my%20power%20went%20out%20%20since%3A2018-01-01%20until%3A2018-01-02&src=typd
Traceback (most recent call last):
  File "/Users/aidancurley/.conda/envs/project5/lib/python3.8/site-packages/GetOldTweets3/manager/TweetManager.py", line 343, in getJsonResponse
    response = opener.open(url)
  File "/Users/aidancurley/.conda/envs/project5/lib/python3.8/urllib/request.py", line 531, in open
    response = meth(req, response)
  File "/Users/aidancurley/.conda/envs/project5/lib/python3.8/urllib/request.py", line 640, in http_response
    response = self.parent.error(
  File "/Users/aidancurley/.conda/envs/project5/lib/python3.8/urllib/request.py", line 569, in error
    return self._call_chain(*args)
  File "/Users/aidancurley/.conda/envs/project5/lib/python3.8/urllib/request.py", line 502, in _call_chain
    result = func(*args)
  File "/Users/aidancurley/.conda

TypeError: object of type 'NoneType' has no len()

In [4]:
df.head(3)

NameError: name 'df' is not defined

In [10]:
len(tweets)

0

In [75]:
df.drop_duplicates().reset_index().drop(columns='index')

,tweets,user
0,Aaaaand my power went out,Tex_tR
1,"Action-packed gameplay, an original cinematic ...",PlayAvengers
2,my power really went out for almost 2 hours th...,samlopezz03
3,My power went out with my finished 3d model le...,Chicken_Nuggie2
4,Okay this is when the power went out in the th...,SumoYokai
...,...,...
7029,I'm only a vehement feminist radical in the da...,beanetry
7030,"""I'm trying real hard to look like I care but ...",Strongisgentle
7031,Folks when OUR President is re-elected (and he...,Freedomfriend64
7032,Anteaters Everyday: Lunch Time Day 3 The subti...,powerJAMM


In [45]:
user[0][0]

'rynertia'

## **Finalized DataFrame**

In [46]:
# This functions will splits the pulled user name into a list of strings in a list
def split(users):
    fin_user = []
    for xi in users:
        for n in xi:
            fin_user.append(n.split(","))
            
    return fin_user

users = split(user) # This is calling the function

hmm = [tweet[0] for sublist in tweets for tweet in sublist] # This gets us the string of the tweet before it was a list of list of lists


fin_user = [''.join(ele) for ele in users] # This gets rid of the [] around the individual stringed usernames
df = pd.DataFrame({'tweets': hmm, 
                   'user': fin_user
                  })

In [47]:
df.head(1)

,tweets,user
0,i breathed so hard into my mic that my houses ...,rynertia


## **Adding Location Using Tweepy**

In [48]:
consumer_key= 'nOj8CQ7XjcNbTS9fFPp6wUNQF'
consumer_secret= '0VtlM5UXl2ZLAL0WMMT9WQ0kHFz4ufKTtn8T1bNVNpvdzYb2Mi'
access_token= '1301516905136771072-TrLtF8u5u0pUBmkSULnErXQcuyVk9i'
access_token_secret= 'hxCKEmbRNE1w0kn0H1owznIQJfuSjDGF3AaZLBCXgqQBO'

In [49]:
auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)

In [50]:
user_locs = [n for n in df['user']]
user_locs[0]

'rynertia'

In [51]:
# the screen name of the user 
location_final = []
for xi in user_locs:
    
    screen_name = xi

    # fetching the user 
    user = api.get_user(screen_name) 

    # fetching the name 
    name = user.name 

    location = user.location

    if location == "": 
        location_final.append("The user has not mentioned their location.") 
    else: 
        location_final.append(location) 
        
    time.sleep(3)

In [52]:
location_final[0]

'The user has not mentioned their location.'

In [53]:
df['location'] = location_final

In [54]:
df

,tweets,user,location
0,i breathed so hard into my mic that my houses ...,rynertia,The user has not mentioned their location.
1,"Action-packed gameplay, an original cinematic ...",PlayAvengers,"San Francisco, CA"
2,my power just went completely out,gldencherry_,she/her
3,my power just went out,goIdentayIor,⅖ || wendy🤍 || 17 || lesbian
4,Today was an eventful day. I got most of my tr...,Boopcosplay,"Leicester, England"
...,...,...,...
678,Just lost power. I hate everything sadKEK http...,NachoOtter,"Michigan, USA"
679,"“In my opinion, the current president is tryin...",FreedomEnslaves,London Town
680,ai dungeon: huh? i thought lost was a girl me:...,guppystail,some dark place
681,I’m so blessed to work with these GKs every we...,ItsJustKelsss1,The user has not mentioned their location.


In [61]:
df['tweets']

0      i breathed so hard into my mic that my houses ...
1      Action-packed gameplay, an original cinematic ...
2                     my power just went completely out 
3                                 my power just went out
4      Today was an eventful day. I got most of my tr...
                             ...                        
678    Just lost power. I hate everything sadKEK http...
679    “In my opinion, the current president is tryin...
680    ai dungeon: huh? i thought lost was a girl me:...
681    I’m so blessed to work with these GKs every we...
682    @TheCatOnFox can you ask. Im a coach and ive w...
Name: tweets, Length: 683, dtype: object

In [2]:

# ================================================================================================================================    
# Now using Tweepy to get user location
# consumer_key= 'nOj8CQ7XjcNbTS9fFPp6wUNQF'
# consumer_secret= '0VtlM5UXl2ZLAL0WMMT9WQ0kHFz4ufKTtn8T1bNVNpvdzYb2Mi'
# access_token= '1301516905136771072-TrLtF8u5u0pUBmkSULnErXQcuyVk9i'
# access_token_secret= 'hxCKEmbRNE1w0kn0H1owznIQJfuSjDGF3AaZLBCXgqQBO'
    
# auth = tw.OAuthHandler(consumer_key, consumer_secret)
# auth.set_access_token(access_token, access_token_secret)
# api = tw.API(auth, wait_on_rate_limit=True)

# user_locs = [n for n in df['user']]

# the screen name of the user 

# location_final = []
# for xi in user_locs:
    
#     screen_name = xi

#     # fetching the user 
#     user = api.get_user(screen_name) 

#     # fetching the name 
#     name = user.name 

#     location = user.location

#     if location == "": 
#         location_final.append("The user has not mentioned their location.") 
#     else: 
#         location_final.append(location) 
        
#     time.sleep(3)
    
# df['location'] = location_final